## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from feature_engine.selection import DropCorrelatedFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance, SelectByShuffling, RecursiveFeatureElimination, RecursiveFeatureAddition, DropConstantFeatures

import lightgbm as lgb
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import CategoricalNB

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import roc_auc_score, confusion_matrix, accuracy_score, classification_report, f1_score

import pickle
import os
import glob
import joblib


In [4]:
pip install feature_engine

Note: you may need to restart the kernel to use updated packages.


In [123]:
pip install imblearn

Note: you may need to restart the kernel to use updated packages.


In [124]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [125]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


## Importing Data

In [2]:
i = 1
frames = []
for name in os.listdir('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data'):
    extents = os.path.splitext('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data'+'/{}'.format(name))[1].lower()
    if extents == '.csv':
        df= pd.read_csv('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data/{}'.format(name))
        frames.append(df)
        print('csv',df.shape)
    elif extents == '.xlsx':
        df= pd.read_excel('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/client_data/{}'.format(name))
        df.columns = df.columns.astype(str)
        frames.append(df)
        print('xlx',df.shape)

result =  pd.concat(frames, axis=0, ignore_index= True)

csv (180, 1060)
csv (244, 1060)
csv (24, 1060)
csv (200, 1060)
csv (302, 1060)
csv (16, 1060)
csv (72, 1060)
csv (244, 1060)
csv (48, 1060)
csv (90, 1060)
csv (314, 1060)
csv (78, 1060)
csv (191, 1060)
csv (310, 1060)
csv (294, 1060)
csv (132, 1059)
csv (224, 1060)
csv (224, 1060)
csv (18, 1060)
csv (236, 1060)
csv (160, 1060)
csv (42, 1060)
csv (246, 1060)
csv (302, 1060)
csv (262, 1060)
csv (162, 1060)
csv (142, 1060)
csv (154, 1060)
csv (256, 1060)
csv (192, 1060)
csv (252, 1060)
csv (78, 1060)
csv (270, 1060)
csv (186, 1060)
csv (29, 1060)
csv (276, 1060)
csv (120, 1060)
csv (80, 1060)
csv (28, 1060)
csv (249, 1060)
csv (120, 1060)
csv (248, 1060)
csv (348, 1060)
csv (290, 1060)
csv (102, 1060)
csv (180, 1060)
csv (110, 1060)
csv (198, 1060)
csv (132, 1060)
csv (222, 1060)
csv (77, 1060)
csv (110, 1060)
csv (10, 1060)
csv (60, 1060)
csv (254, 1060)
csv (36, 1060)
xlx (680, 1059)
csv (210, 1060)
csv (326, 1060)
csv (90, 1060)
csv (180, 1060)
csv (230, 1060)
csv (250, 1060)
csv (302,

In [3]:
result.head()

,,L*,a*,b*,X,Y,Z,Dominant Wavelenght,Whiteness,Purity,...,979.335,980.054,980.772,981.491,982.209,982.928,983.646,984.364,985.082,Unnamed: 1059
0,W_Calibration,100.0000,0.000000,0.00000,95.0422,100.0000,108.8760,565.016,99.9909,0.000017,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.00,NaN
1,F_Calibration,100.0000,0.000000,0.00000,108.1040,100.0000,39.2965,491.070,99.9909,0.000170,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.00,NaN
2,W_Calibration,100.0000,0.000000,0.00000,95.0422,100.0000,108.8760,565.016,99.9909,0.000017,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.00,NaN
3,F_Calibration,100.0000,0.000000,0.00000,108.1040,100.0000,39.2965,491.070,99.9909,0.000170,...,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.0000,100.00,NaN
4,W_3566A2/33_H_Faint_0.30_SI1,99.7016,-0.840692,1.97141,93.8345,99.2302,104.8660,569.412,90.5883,0.017551,...,97.4079,97.6368,98.0312,97.8384,97.8231,-33.8442,-71.7545,-33.0428,-1563.49,NaN


## Data PrePrpcessing

In [4]:
result.rename(columns = {' ': 'Output'}, inplace=True)

In [5]:
result_3 =result.loc[:,'Output']
result_2= result.loc[:,'300.377':'700.577']
result_1= result.iloc[:,1:14]


In [6]:
df = pd.concat([result_2, result_3], axis=1)

In [7]:
df.shape

(15162, 524)

In [11]:
# df.to_excel('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/whole_data.xlsx')

In [8]:
df_without_calb = df[~df['Output'].str.contains(('Calibration|CALIBARTION|CALIBARATION'),case=False, regex=True, na = False)]
df_without_calb.reset_index(drop=True)
df_without_calb['w/f']= df_without_calb['Output'].str.split('_').str[0]

In [10]:
df_with_w = df_without_calb[df_without_calb['w/f'] == 'W']

In [273]:
df_with_w.size.astype(str)

'2032800'

In [12]:
df_with_w['quality'] = df_with_w['Output'].str.split('_').str[3]
df_with_w['quality'].replace({'lb':'Faint','Benth':'None','brown':'None','Pink':'None', 'LB':'None','LB ':'None', 'Faint YELLOW':'Faint', 'Faint YELLOW':'Faint', 'Faint GREEN':'Faint','  ':'None', 'BROWN':'Faint', 'Lblack':'Faint', 'lb ':'None','Black':'None'}, inplace = True)

In [13]:
df_with_w['color'] = df_with_w['Output'].str.split('_').str[2]
df_with_w['color'].replace({'d':'D','H/BLK':'H','E/LB':'E','H/LB':'H','J/LB':'J','I/LB':'I','J ':'J','F/LB':'F','G/LB':'G','E LB':'E', ' K':'K', 'G LB ':'G', 'H ':'H'}, inplace = True)


In [14]:
df_with_w.drop((df_with_w[df_with_w.color.str.contains('\s', regex=True)].index|df_with_w[df_with_w['color'] == ''].index|df_with_w[df_with_w['quality'].str.contains('\s\W', regex=True)].index|df_with_w[df_with_w['color'].str.contains('\s\W', regex=True)].index|df_with_w[df_with_w['quality'].str.contains(('^\s'), regex=True)].index), inplace=True)

In [15]:
df_with_w.drop(columns=['Output','w/f'], inplace=True)

In [16]:
df_with_w = df_with_w[~(df_with_w['quality'] == 'Strong')]

In [342]:
df_with_w.to_csv('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/df_w_1_08.csv')

In [19]:
df_with_w.quality.value_counts()

None           2818
Faint          1148
Very Strong     656
Medium          377
Name: quality, dtype: int64

In [23]:
df_visu = df_with_w.copy()
df_visu.reset_index(drop=True, inplace=True)

In [40]:
df_visu['300.377'].astype('float').min()

65.1567

## Reomoving Outliers

In [39]:
# df_visu.drop(index = df_visu['300.377'].ast/ype('float').idxmax(), inplace=True)
# df_visu.drop(index=df_visu['300.377'].astype('float').idxmin(), inplace=True)

In [42]:
df_visu

,300.377,301.162,301.946,302.731,303.515,304.299,305.083,305.867,306.652,307.435,...,695.339,696.087,696.836,697.584,698.332,699.081,699.829,700.577,quality,color
0,93.9613,93.5905,93.5144,93.5858,94.0028,93.9788,94.2287,93.9844,94.2687,94.7686,...,99.3325,99.3291,99.3268,99.3363,99.3128,99.3191,99.3499,99.3758,Faint,H
1,95.5023,95.4314,95.2679,95.4398,96.0739,96.0221,96.2585,96.0292,96.0922,96.1002,...,100.3670,100.3130,100.2990,100.2630,100.2110,100.1920,100.1960,100.2210,None,J
2,96.7677,96.5512,96.1780,96.4226,96.8096,96.9811,96.8336,96.8316,97.0489,96.5584,...,100.5560,100.5590,100.5620,100.5480,100.5370,100.5330,100.5110,100.4660,None,H
3,92.1969,91.9748,91.8531,92.1156,92.3446,92.4588,92.2696,92.1126,92.2204,91.8990,...,99.0999,99.1498,99.1580,99.1579,99.2034,99.2142,99.2092,99.1851,None,J
4,97.9493,97.5832,97.8770,97.8452,98.2188,98.1894,98.1268,98.2943,97.7774,97.8066,...,101.1530,101.1980,101.2170,101.2350,101.2070,101.2170,101.2320,101.2150,None,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,102.0850,102.2880,102.4110,102.7420,102.2430,102.2220,102.2150,102.2560,102.2080,102.2340,...,103.4040,103.4020,103.4040,103.4300,103.4020,103.4220,103.4880,103.4690,None,E
4995,97.9494,98.1799,98.3628,98.0291,98.3103,98.4854,98.0282,97.5581,97.6816,97.6634,...,99.8242,99.8488,99.8604,99.8668,99.8569,99.8530,99.8555,99.8538,Faint,F
4996,96.4612,96.7232,96.9706,96.8789,96.7985,97.1090,97.2549,97.4836,97.5302,97.3204,...,99.9035,99.9315,99.9107,99.9335,99.9170,99.9572,99.9600,99.9568,None,G
4997,97.1866,97.0617,97.2634,97.6980,97.2498,97.3879,97.5460,97.7210,98.4481,98.3731,...,100.8130,100.8300,100.8580,100.8570,100.8590,100.8910,100.9130,100.9130,None,D


In [43]:
df_visu.to_csv('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/df_train_2_08.csv')

In [114]:
df_train = pd.read_csv('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/df_train_2_08.csv')

In [115]:
df_train.drop(columns='Unnamed: 0', inplace=True)

# Model Implementation

In [65]:
train = pd.read_csv('/home/pankaj_v/Documents/Data_analysis_dixit/dATA/Processed_data/Feature_selected/x_se_shuff_1_08.csv')

In [91]:
train.drop(columns='Unnamed: 0', inplace=True)

In [86]:
df_train.drop(columns='color', inplace=True)

In [111]:
df_X_train = pd.get_dummies(df_train)

### Train Test Split

In [112]:
X = df_X_train[train.columns]

In [116]:
Y = df_train.iloc[:,-1]

In [141]:
Y.value_counts().keys()

Index(['D', 'G', 'F', 'E', 'H', 'I', 'J', 'K', 'L', 'M'], dtype='object')

In [118]:
X.shape, Y.shape

((4993, 264), (4993,))

In [120]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state =42, stratify = Y)

In [487]:
X_train_cv, X_test_cv, Y_train_cv, Y_test_cv = train_test_split(X_train,Y_train, test_size = 0.2, stratify = Y_train)

In [121]:
skf = StratifiedKFold(n_splits=5, shuffle=True)

In [142]:
balanced_ = class_weight.compute_class_weight(class_weight='balanced', classes=Y.value_counts().keys(),y=Y)
balanced = dict(zip(df_visu['color'].value_counts().keys(),balanced_))


In [143]:
balanced

{'D': 0.6171817058096415,
 'G': 0.6187112763320942,
 'F': 0.682103825136612,
 'E': 0.823927392739274,
 'H': 0.8491496598639455,
 'I': 0.9565134099616859,
 'J': 1.0358921161825727,
 'K': 1.4265714285714286,
 'L': 7.8015625,
 'M': 15.13030303030303}

### ExtraTreesClassifier

In [144]:
et = ExtraTreesClassifier( criterion = 'entropy',
                            class_weight=balanced, 
                            random_state=42, 
                            warm_start=True, 
                            max_samples=None, 
                            bootstrap=True,
                            max_depth=15,
                            )
grid_values = {
    'n_estimators':[1000]
              }
              
et_grid = RandomizedSearchCV(et,param_distributions= grid_values,
    cv = 5, scoring='f1_weighted', random_state=42)

In [145]:
et_grid.fit(X_train, Y_train)

RandomizedSearchCV(cv=5,
                   estimator=ExtraTreesClassifier(bootstrap=True,
                                                  class_weight={'D': 0.6171817058096415,
                                                                'E': 0.823927392739274,
                                                                'F': 0.682103825136612,
                                                                'G': 0.6187112763320942,
                                                                'H': 0.8491496598639455,
                                                                'I': 0.9565134099616859,
                                                                'J': 1.0358921161825727,
                                                                'K': 1.4265714285714286,
                                                                'L': 7.8015625,
                                                                'M': 15.13030303030303},
                              

In [158]:
et_grid.best_estimator_

ExtraTreesClassifier(bootstrap=True,
                     class_weight={'D': 0.6171817058096415,
                                   'E': 0.823927392739274,
                                   'F': 0.682103825136612,
                                   'G': 0.6187112763320942,
                                   'H': 0.8491496598639455,
                                   'I': 0.9565134099616859,
                                   'J': 1.0358921161825727,
                                   'K': 1.4265714285714286, 'L': 7.8015625,
                                   'M': 15.13030303030303},
                     criterion='entropy', max_depth=15, n_estimators=1000,
                     random_state=42, warm_start=True)

In [159]:
pd.DataFrame(et_grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,5.833772,0.352211,0.198739,0.004792,1000,{'n_estimators': 1000},0.506344,0.547327,0.513532,0.493836,0.510107,0.514229,0.017838,1


In [168]:
lst_accu_stratified = []
lst_accu_stratified_train = []


for train_index, test_index in skf.split(X_train, Y_train):
  x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
  y_train_fold, y_test_fold = Y.iloc[train_index], Y.iloc[test_index]

  clf_ = et
  clf_.fit(x_train_fold, y_train_fold)
  y_train_pred = clf_.predict(x_train_fold)
  Y_pred = clf_.predict(x_test_fold)
  print(classification_report(y_test_fold, Y_pred))

  lst_accu_stratified.append(metrics.f1_score(y_test_fold,Y_pred,average='weighted'))
  lst_accu_stratified_train.append(clf_.score(x_train_fold,y_train_fold))

print(f"train_accuracy={lst_accu_stratified_train}")
print(f"f1_score={lst_accu_stratified}")

print(f" over all accuracy train_accuracy={np.mean(lst_accu_stratified_train)*100}")
print(f"f1_score_mean = {np.mean(lst_accu_stratified)*100}")

              precision    recall  f1-score   support

           D       0.93      0.92      0.92       109
           E       0.85      0.89      0.87       102
           F       0.91      0.87      0.89       128
           G       0.83      0.87      0.85       134
           H       0.91      0.80      0.85        99
           I       0.81      0.79      0.80        75
           J       0.84      0.97      0.90        79
           K       0.95      0.92      0.93        61
           L       0.88      1.00      0.93         7
           M       1.00      0.80      0.89         5

    accuracy                           0.88       799
   macro avg       0.89      0.88      0.88       799
weighted avg       0.88      0.88      0.88       799

              precision    recall  f1-score   support

           D       0.97      0.98      0.97       134
           E       0.94      0.91      0.93        91
           F       0.89      0.88      0.88       108
           G       0.87 

In [162]:
y_preds  = et.predict(X_test)

In [125]:
print(classification_report(Y_test, y_preds))

              precision    recall  f1-score   support

           D       0.91      0.91      0.91       162
           E       0.81      0.79      0.80       121
           F       0.81      0.68      0.74       146
           G       0.66      0.88      0.76       162
           H       0.85      0.67      0.75       118
           I       0.80      0.80      0.80       104
           J       0.82      0.83      0.82        96
           K       0.82      0.87      0.85        70
           L       0.75      0.46      0.57        13
           M       1.00      0.57      0.73         7

    accuracy                           0.80       999
   macro avg       0.82      0.75      0.77       999
weighted avg       0.81      0.80      0.80       999



In [410]:
joblib.dump(
        clf_,  
        os.path.join('/home/pankaj_v/Documents/Data_analysis_dixit/Models', f"f1_{round(np.mean(lst_accu_stratified)*100,2)}_folds.pkl") 
    )

['/home/pankaj_v/Documents/Data_analysis_dixit/Models/f1_89.05_folds.pkl']